this notebook is to show that for certain cases, cls=0 gives exactly the same result as a regualr maskcnn.

I will use configurations in <https://github.com/leelabcnbc/thesis-yimeng-v2/blob/00b0ae57793923722941525658927954cc9c9961/results_processed/yuanyuan_8k_a_3day/data-driven_vs_transfer/baseline_comparison.ipynb>

In [1]:
# this is mostly a copy of <https://github.com/leelabcnbc/thesis-yimeng-v2/blob/master/scripts/training/yuanyuan_8k_a_3day/maskcnn_polished/master.py>
from os.path import join

import numpy as np

import torch

from thesis_v2 import dir_dict

from thesis_v2.data.prepared.yuanyuan_8k import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished.builder import (
    gen_maskcnn_polished,
    load_modules as load_modules_maskcnn_polished
)




In [2]:
def train_one_maskcnn_polished(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    bn_before_act,
    bn_after_fc
):
    
    load_modules_maskcnn_polished()
    datasets = get_data('a', 200, input_size, ('042318', '043018', '051018'), scale=0.5,
                        seed=split_seed)

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished(input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    bn_before_act=bn_before_act,
                                    bn_after_fc=bn_after_fc,
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    # let's load back the old sigularity image's data.
    init_data = torch.load(join(dir_dict['models'],
                                f'debug/compare_cls0_with_regular_maskcnn/regular/{model_seed}',
                                'init.pth'
                               )
                          )
    
    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/compare_cls0_with_regular_maskcnn_20200106image/regular/{model_seed}',
        show_every=100000000,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params = {
            'init_model_state_dict': init_data['model'],
            'init_optimizer_state_dict': init_data['optimizer'],
        }
    )
    
    return result['stats_best']['stats']['test']['corr_mean']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 2,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 'legacy',
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'bn_before_act': True,
        'loss_type': 'mse',
        'smoothness': 0.000005,
        'smoothness_name': '0.000005',
        'scale': 0.01,
        'scale_name': '0.01',
        'bn_after_fc': False,
        'input_size': 50,
    }
}

print(train_one_maskcnn_polished(**maskcnn_param_regular))

num_param 20493
========starting phase 1/3==========
val metric init {'loss': 0.14660925567150115, 'loss_no_reg': 0.14377041161060333, 'corr': [0.031199224767873273, -0.028956205931587507, -0.08372151462837896, 0.043572884765579044, 0.010826193942577465, 0.034604413618673266, 0.013517815689188754, -0.046275859086092726, 0.05008616903475993, -0.006272808372306695, -0.052915693451366944, -0.04240224542449176, 0.10408493550982435, 0.15733014245519714, 0.03082162794788332, 0.10673673418917723, -0.043764611933623285, 0.09467284143041245, 0.031702008296795633, -0.022610198279625973, 0.09690055210741044, -0.07490630155887545, 0.023630931259834505, 0.008976398043703784, 0.015267047152058424, 0.0004162525972721473, -0.0011892501544335903, -0.05566246996071895, 0.07597629814964535, 0.011358189812904406, 0.023784924307746896, 0.059023888481577254, 0.036599878670991084, 0.0986183674176384, 0.03713361297407748, -0.1166849180332873, -0.0005335708040295987, 0.07098491979209895, -0.1318473546234682, -

early stopping after epoch 900 metric 0.10791107267141342
========end phase 2/3==========
========starting phase 3/3==========
for grp of sz 11, lr from 0.000333 to 0.000111
val metric init {'loss': 0.11172574162483215, 'loss_no_reg': 0.10791107267141342, 'corr': [0.26907584216757297, 0.4972165428473387, 0.6109314998120043, 0.7414386857436739, 0.5079044704056711, 0.5138336152252034, 0.5113629541453009, 0.6259727374004935, 0.6829376008276378, 0.7230403032612664, 0.699924794617514, 0.5379171230172088, 0.6018746512137507, 0.6518044974845967, 0.6592416265137439, 0.6213237187673513, 0.7022948195388499, 0.6059302715433335, 0.5908453721937335, 0.6889189858993571, 0.47664369658468475, 0.47012688752297, 0.3458699950293782, 0.11180851201794775, 0.762359889372259, 0.6167443971534261, 0.5724390635430857, 0.3695816049979433, 0.6546511406526672, 0.5549284596278767, 0.38551123520452224, 0.5691630359927065, 0.5284095767505335, 0.5711214785187766, 0.6200568153408748, 0.5254140687619521, 0.6116017734668

In [4]:
from thesis_v2.models.maskcnn_polished_with_local_pcn.builder import (
    gen_maskcnn_polished_with_local_pcn,
    load_modules as load_modules_maskcnn_polished_with_local_pcn
)

def train_one_maskcnn_polished_with_local_pcn(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    bn_before_act,
    bn_after_fc,
    
    
    pcn_bn,
    pcn_bn_post,
    pcn_bypass,
    pcn_cls,
    pcn_final_act,
    pcn_no_act,
    pcn_bias,
):
    
    load_modules_maskcnn_polished_with_local_pcn()
    datasets = get_data('a', 200, input_size, ('042318', '043018', '051018'), scale=0.5,
                        seed=split_seed)

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': datasets[1],
        'X_val': datasets[2].astype(np.float32),
        'y_val': datasets[3],
        'X_test': datasets[4].astype(np.float32),
        'y_test': datasets[5],
    }

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_local_pcn(
            input_size=input_size_cnn,
            num_neuron=n,
            out_channel=out_channel,
            kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
            kernel_size_l23=3,
            act_fn=act_fn,
            pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
            pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
            num_layer=num_layer,
            bn_before_act=bn_before_act,
            bn_after_fc=bn_after_fc,


            pcn_bn=pcn_bn,
            pcn_bn_post=pcn_bn_post,
            pcn_bypass=pcn_bypass,
            pcn_cls=pcn_cls,
            pcn_final_act=pcn_final_act,
            pcn_no_act=pcn_no_act,
            pcn_b0_init=1.0,
            pcn_tied=False,
            pcn_bias=pcn_bias,
        )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    # let's load back the old sigularity image's data.
    init_data = torch.load(join(dir_dict['models'],
                                f'debug/compare_cls0_with_regular_maskcnn/local_pcn/{pcn_cls}/{model_seed}',
                                'init.pth'
                               )
                          )

    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets,
        key=f'debug/compare_cls0_with_regular_maskcnn_20200106image/local_pcn/{pcn_cls}/{model_seed}',
        show_every=100000000,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params = {
            'init_model_state_dict': init_data['model'],
            'init_optimizer_state_dict': init_data['optimizer'],
        }
    )
    
    return result['stats_best']['stats']['test']['corr_mean']

In [5]:
maskcnn_param_local_pcn_cls0 = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'bn_before_act': True,
        'loss_type': 'mse',
        'smoothness': 0.000005,
        'smoothness_name': '0.000005',
        'scale': 0.01,
        'scale_name': '0.01',
        'bn_after_fc': False,
        'input_size': 50,
        'pcn_bn': True,
        'pcn_bn_post': False,
        'pcn_bypass': False,
        'pcn_cls': 0,
        'pcn_final_act': True,
        
        # this can ensure we get same stuff.
        'pcn_no_act': True,
        'pcn_bias': False,
    }
}

print(train_one_maskcnn_polished_with_local_pcn(**maskcnn_param_local_pcn_cls0))

num_param 20493
========starting phase 1/3==========
val metric init {'loss': 0.14660925567150115, 'loss_no_reg': 0.14377041161060333, 'corr': [0.031199224767873273, -0.028956205931587507, -0.08372151462837896, 0.043572884765579044, 0.010826193942577465, 0.034604413618673266, 0.013517815689188754, -0.046275859086092726, 0.05008616903475993, -0.006272808372306695, -0.052915693451366944, -0.04240224542449176, 0.10408493550982435, 0.15733014245519714, 0.03082162794788332, 0.10673673418917723, -0.043764611933623285, 0.09467284143041245, 0.031702008296795633, -0.022610198279625973, 0.09690055210741044, -0.07490630155887545, 0.023630931259834505, 0.008976398043703784, 0.015267047152058424, 0.0004162525972721473, -0.0011892501544335903, -0.05566246996071895, 0.07597629814964535, 0.011358189812904406, 0.023784924307746896, 0.059023888481577254, 0.036599878670991084, 0.0986183674176384, 0.03713361297407748, -0.1166849180332873, -0.0005335708040295987, 0.07098491979209895, -0.1318473546234682, -

early stopping after epoch 900 metric 0.10791107267141342
========end phase 2/3==========
========starting phase 3/3==========
for grp of sz 11, lr from 0.000333 to 0.000111
val metric init {'loss': 0.11172574162483215, 'loss_no_reg': 0.10791107267141342, 'corr': [0.26907584216757297, 0.4972165428473387, 0.6109314998120043, 0.7414386857436739, 0.5079044704056711, 0.5138336152252034, 0.5113629541453009, 0.6259727374004935, 0.6829376008276378, 0.7230403032612664, 0.699924794617514, 0.5379171230172088, 0.6018746512137507, 0.6518044974845967, 0.6592416265137439, 0.6213237187673513, 0.7022948195388499, 0.6059302715433335, 0.5908453721937335, 0.6889189858993571, 0.47664369658468475, 0.47012688752297, 0.3458699950293782, 0.11180851201794775, 0.762359889372259, 0.6167443971534261, 0.5724390635430857, 0.3695816049979433, 0.6546511406526672, 0.5549284596278767, 0.38551123520452224, 0.5691630359927065, 0.5284095767505335, 0.5711214785187766, 0.6200568153408748, 0.5254140687619521, 0.6116017734668

In [6]:
# exactly match.

In [7]:
# results are very close to that in

# compare_cls0_with_regular_maskcnn.ipynb

# for the final 'corr' in `test metric`,

# one image has (mean, std) of (0.5203765276235651, 0.13155096222934082)
# another has (0.5220329388032986, 0.13231455096159658)
#
# pearsonr gives (0.9985405844539036, 2.2711330866032455e-99)